In [ ]:
import wisardpkg as wp
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


In [ ]:
MNIST = fetch_openml('mnist_784', version=1, as_frame=False)

X = MNIST.data
y = MNIST.target
X_reshaped = X.reshape(-1, 28, 28)
X_reshaped[0].shape
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

In [ ]:
class RandomKernelCanvas():
    def __init__(self, shape : tuple, numberOfKernels : int, bitsByKernel : int = 3, activationDegree : float = 0.07):
        self.numberOfKernels = numberOfKernels
        self.bitsByKernel = bitsByKernel
        self.activationDegree = activationDegree
        self.shape = shape
        self.closestKernel: dict[tuple[int, int], int] = {}
        
        self.makeKernel()
        self.findClosestKernels()
        
    def squareDistance(self, x, y):
        return np.sum(np.pow(x - y, 2))
        
    def makeKernel(self):
        points = np.random.choice(np.prod(self.shape), size=self.numberOfKernels, replace=False)
        self.kernel_points = [(point // self.shape[1], point % self.shape[1]) for point in points]
        
    def findClosestKernels(self):
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                self.closestKernel[(i, j)] = int(self.findClosestKernel((i, j)))
        
    def findClosestKernel(self, x):
        distances = [self.squareDistance(x, np.array(self.kernel_points[i])) for i in range(self.numberOfKernels)]
        return np.argmin(distances)
        
    def showCanvas(self):
        canvas = np.ones((*self.shape, 3))  # All white squares, RGB

        # Paint the kernel points red
        for row, col in self.kernel_points:
            canvas[row, col] = [1, 0, 0]  # Red color (R=1, G=0, B=0)
        
        fig, ax = plt.subplots()
        ax.imshow(canvas, vmin=0, vmax=1)
        
         # Add numbers to each square
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                ax.text(j, i, str(self.findClosestKernel((i, j))), 
                        va='center', ha='center', color='black', fontsize=6)

        plt.show()
        
    def transform(self, data) -> list:
        if data.shape != self.shape:
            raise ValueError(f"Input data shape {data.shape} does not match expected shape {self.shape}")
        ret = [0 for _ in range(self.numberOfKernels)]
        # ret = np.zeros(self.numberOfKernels, dtype=int)
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                kernel_index = self.closestKernel[(i, j)]
                if(data[i, j] > self.activationDegree):
                    ret[kernel_index] = 1
        return ret
    
    def showTransformed(self, TransformedData):
        if len(TransformedData) != self.numberOfKernels:
            raise ValueError(f"Transformed data length {len(TransformedData)} does not match number of kernels {self.numberOfKernels}")
        
        canvas = np.ones((*self.shape, 3))  # All white squares, RGB

        
        
        # paint the activated kernels
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                if(TransformedData[self.closestKernel[(i, j)]]):
                    canvas[i, j] = [0, 0, 0]

        # Paint the kernel points red
        for row, col in self.kernel_points:
            canvas[row, col] = [1, 0, 0]  # Red color (R=1, G=0, B=0)
            
        fig, ax = plt.subplots()
        ax.imshow(canvas, vmin=0, vmax=1)
        plt.show()
                    
                

In [ ]:
# Create a 28x28 Gaussian distribution centered in the grid
x = np.linspace(0, 27, 28)
y = np.linspace(0, 27, 28)
xv, yv = np.meshgrid(x, y)
mean = [13.5, 13.5]
cov = [[30, 0], [0, 30]]  # Adjust variance for spread

gaussian = np.exp(-((xv - mean[0])**2 + (yv - mean[1])**2) / (2 * cov[0][0]))
gaussian /= gaussian.max()  # Normalize to [0, 1]
plt.imshow(gaussian, cmap='viridis')
plt.colorbar()
plt.title("28x28 Gaussian Distribution")
plt.axis('off')
plt.show()

prob_matrix = gaussian / gaussian.sum()
flattened_prob_matrix = prob_matrix.flatten()
prob_matrix.sum()

In [ ]:
class CenteredKernelCanvas(RandomKernelCanvas):
    def __init__(self, shape : tuple, numberOfKernels : int, bitsByKernel : int = 3, activationDegree : float = 0.07):
        RandomKernelCanvas.__init__(self, shape, numberOfKernels, bitsByKernel, activationDegree)
        
    def makeKernel(self, weights=flattened_prob_matrix):
        flat_size = np.prod(self.shape)
        weights = np.array(weights)
        weights = weights / weights.sum()  # Normalize
        points = np.random.choice(flat_size, size=self.numberOfKernels, replace=False, p=weights)
        self.kernel_points = [(point // self.shape[0], point % self.shape[1]) for point in points]
        
    def mutate(self, mutationFactor=0.1):
        selected_points = np.random.choice(len(self.kernel_points), size=int(self.numberOfKernels * mutationFactor), replace=False)
        directions = [[-1, 0], [1, 0], [0, -1], [0, 1]]  # up, down, left, right
        for i in range(len(selected_points)):
            row, col = self.kernel_points[i]
            direction = np.random.choice(directions)
            new_row, new_col = row + direction[0], col + direction[1]
            if (new_row, new_col) not in self.kernel_points:
                self.kernel_points[i] = (new_row, new_col)
            
        self.findClosestKernels()

In [ ]:
def accuracy(y_pred, y_target):
  return np.where(y_pred == y_target, 1, 0).sum()/len(y_target)

In [ ]:
from multiprocessing import Pool, cpu_count


def evaluate_individual(canvas):
    train_points = [canvas.transform(x) for x in X_train]
    test_points = [canvas.transform(x) for x in X_test]
    train_ds = wp.DataSet(train_points, y_train)
    test_ds = wp.DataSet(test_points, y_test)
    model = wp.Wisard(16, verbose=False)
    model.train(train_ds)
    pred = model.classify(test_ds)
    return accuracy(pred, y_test)

num_generations = 10
mutation_rate = 0.1

# model = wp.Wisard(16, verbose=False)
genePool = [CenteredKernelCanvas((28, 28), 128, bitsByKernel=1, activationDegree=0.07) for _ in range(cpu_count())]

for generation in range(num_generations):
    print(f"Generation {generation} started.")
    
    with Pool(cpu_count()) as pool:
        fitness = pool.map(evaluate_individual, (genePool))
    print(f"Generation {generation}, best accuracy: {max(fitness):.4f}")

    # Select top 50% individuals
    survivalChance = 0.5
    sorted_indices = np.argsort(fitness)[::-1]
    survivors : list[CenteredKernelCanvas] = [genePool[i] for i in sorted_indices[:len(genePool)*survivalChance]]

    # Reproduce: copy survivors and mutate
    offspring = survivors.copy()
    for i in offspring:
        i.mutate(mutationFactor=mutation_rate)
        
    genePool = survivors + offspring



In [1]:
from multiprocessing import cpu_count

cpu_count()

8